In [42]:
# improt the library and set the data set
import tensorflow as tf
import numpy      as np
from sklearn import datasets
np.set_printoptions(precision=3,suppress=True)
np.random.seed(789)
tf.set_random_seed(678)
sess = tf.InteractiveSession()

C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\client\session.py:1711: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


In [19]:
# create data and basic reviews
data,label = datasets.make_blobs(9000,3)
s,U,V      = tf.linalg.svd(data,full_matrices=False)
s = s.eval(); V = V.eval();U = U.eval();
print('Original Data Shape: ', data.shape)
print('U Data Shape: ', U.shape)
print('s Data Shape: ', s.shape)
print('V Data Shape: ', V.shape)
original_data = (U @ tf.diag(s) @ tf.transpose(V)).eval()
print("Is it same with original data? : ", np.allclose(original_data,data))

Original Data Shape:  (9000, 3)
U Data Shape:  (9000, 3)
s Data Shape:  (3,)
V Data Shape:  (3, 3)
Is it same with original data? :  True


In [32]:
# see if orthogonal property hold
print(' Are they all Identity Matrix? ', np.allclose(np.eye(U.shape[1]), U.T @ U ))
print(' Are they all Identity Matrix? ', np.allclose(np.eye(V.shape[0]), V   @ V.T ))
print(' Are they all Identity Matrix? ', np.allclose(np.eye(V.shape[1]), V.T @ V ))

since_not_full_matrice = U @ U.T
_,U_full,_= tf.linalg.svd(data,full_matrices=True)
U_full = U_full.eval()

print(' This is not an identity matrix: ', np.allclose(np.eye(since_not_full_matrice.shape[0]), since_not_full_matrice ))
print(' This is an identity matrix: ', np.allclose(np.eye(since_not_full_matrice.shape[0]), U_full@U_full.T ))

 Are they all Identity Matrix?  True
 Are they all Identity Matrix?  True
 Are they all Identity Matrix?  True
 This is not an identity matrix:  False
 This is an identity matrix:  True


In [36]:
# is inverse true? 
print('Is inverse same as transpose? : ', np.allclose(np.linalg.inv(U_full),U_full.T))
print('Is inverse same as transpose? : ', np.allclose(np.linalg.inv(V),V.T))

Is inverse same as transpose? :  True
Is inverse same as transpose? :  True


In [54]:
# orthogonal compliment
# https://github.com/statsmodels/statsmodels/issues/3039
def orthogonal_complement(x, normalize=False, threshold=1e-15):
    """Compute orthogonal complement of a matrix

    this works along axis zero, i.e. rank == column rank,
    or number of rows > column rank
    otherwise orthogonal complement is empty

    TODO possibly: use normalize='top' or 'bottom'

    """
    x = np.asarray(x)
    r, c = x.shape
    if r < c:
        import warnings
        warnings.warn('fewer rows than columns', UserWarning)

    # we assume svd is ordered by decreasing singular value, o.w. need sort
    s, v, d = np.linalg.svd(x)
    rank = (v > threshold).sum()

    oc = s[:, rank:]

    if normalize:
        k_oc = oc.shape[1]
        oc = oc.dot(np.linalg.inv(oc[:k_oc, :]))
    return oc

orth_u     = orthogonal_complement(U)
orth_u_and_u1 = U.T @ orth_u
orth_u_and_u2 = orth_u.T @ U
print('Shape of Result: ',orth_u_and_u1.shape)
print('Are they all close to zero? : ',np.allclose(orth_u_and_u1,0.0) )

print('Shape of Result: ',orth_u_and_u2.shape)
print('Are they all close to zero? : ',np.allclose(orth_u_and_u2,0.0) )

orth_u_i = orth_u @ orth_u.T
print()


Shape of Result:  (3, 8997)
Are they all close to zero? :  True
Shape of Result:  (8997, 3)
Are they all close to zero? :  True
